In [1]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

# Load the dataset
df = pd.read_csv("CleanedData/player_stats_cleaned.csv")

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Row([
        dbc.Col(html.Div([
            dcc.Dropdown(
                id='team-dropdown',
                options=[{'label': team, 'value': team} for team in df['team'].dropna().unique()],
                multi=True,
                placeholder="Filter by Team",
            )
        ], style={'margin': '10px'}), width=4),

        dbc.Col(html.Div([
            dcc.Dropdown(
                id='player-dropdown',
                options=[{'label': player, 'value': player} for player in df['player'].dropna().unique()],
                multi=True,
                placeholder="Select Players",
            )
        ], style={'margin': '10px'}), width=8),
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id='scatter-plot'), width=12)
    ])
])

@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('team-dropdown', 'value'),
     Input('player-dropdown', 'value')]
)
def update_scatter_plot(selected_teams, selected_players):
    # Filter data based on selections
    filtered_df = df
    if selected_teams:
        filtered_df = filtered_df[filtered_df['team'].isin(selected_teams)]
    if selected_players:
        filtered_df = filtered_df[filtered_df['player'].isin(selected_players)]
    
    # Generate the scatter plot
    fig = px.scatter(
        filtered_df,
        x='goals',
        y='assists',
        color='team',  # Color by team
        hover_name='player',  # Show player name on hover
        title='Goals vs. Assists for Selected Players and Teams'
    )
    
    fig.update_layout(transition_duration=500)
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8052)

